In [2]:
import torch
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2

In [3]:
#load model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../app/best.pt')
model.eval()

Using cache found in /home/denis/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-25 Python-3.10.7 torch-1.13.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [4]:
#csv that contains data about all images+bboxes from mdv4(but we didn't use it)
#it was processed and saved in model.ipynb
res_path = '../data/processed_test.csv'
images = pd.read_csv(res_path, sep='\t', index_col=0)
sequences = images['seq_id'].unique()
images

,height,id,seq_id,location,width,...,file_name,seq_frame_num,seq_num_frames,sub_location,box
0,1024,8b31d3be-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002,20,1280,...,8b31d3be-21bc-11ea-a13a-137349068a90.jpg,0,10,NaN,"[[0, 0.55, 0.558, 0.438]]"
1,1024,8cf202be-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002,20,1280,...,8cf202be-21bc-11ea-a13a-137349068a90.jpg,1,10,NaN,"[[0, 0.557, 0.684, 0.431]]"
2,1024,8a87e62e-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002,20,1280,...,8a87e62e-21bc-11ea-a13a-137349068a90.jpg,2,10,NaN,"[[0, 0.418, 0.712, 0.581]]"
3,1024,8e6994f4-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002,20,1280,...,8e6994f4-21bc-11ea-a13a-137349068a90.jpg,3,10,NaN,"[[0, 0.402, 0.673, 0.584]]"
4,1024,948b29e2-21bc-11ea-a13a-137349068a90,a91ebc18-0cd3-11eb-bed1-0242ac1c0002,20,1280,...,948b29e2-21bc-11ea-a13a-137349068a90.jpg,4,10,NaN,"[[0, 0.547, 0.693, 0.439]]"
...,...,...,...,...,...,...,...,...,...,...,...
60024,1024,981aabf0-21bc-11ea-a13a-137349068a90,a91b0b90-0cd3-11eb-bed1-0242ac1c0002,5,1280,...,981aabf0-21bc-11ea-a13a-137349068a90.jpg,0,5,1.0,"[[0.023, 0.566, 0.396, 0.217]]"
60025,1024,971d8466-21bc-11ea-a13a-137349068a90,a91b0b90-0cd3-11eb-bed1-0242ac1c0002,5,1280,...,971d8466-21bc-11ea-a13a-137349068a90.jpg,1,5,1.0,"[[0.23, 0.576, 0.372, 0.184]]"
60026,1024,90f05d20-21bc-11ea-a13a-137349068a90,a91b0b90-0cd3-11eb-bed1-0242ac1c0002,5,1280,...,90f05d20-21bc-11ea-a13a-137349068a90.jpg,2,5,1.0,"[[0.398, 0.565, 0.345, 0.187]]"
60027,1024,903c3ebc-21bc-11ea-a13a-137349068a90,a91b0b90-0cd3-11eb-bed1-0242ac1c0002,5,1280,...,903c3ebc-21bc-11ea-a13a-137349068a90.jpg,3,5,1.0,"[[0.535, 0.557, 0.341, 0.192]]"


In [8]:
#to disable warnings of pandas assignment
pd.options.mode.chained_assignment = None
def predict_bboxes(img):
    """
    predicting bboxes for all images using yolo model
    :param img: list[Image.Image]
    :return: list[pd.DatafFrame]
    """
    def make_bbox(df):
        """
        filter bbox by confidence
        :param df: pd.DataFrame
        :return: dataframe with bbox
        """
        filtered_df = df[df['confidence'] > 0.3]
        bbox = filtered_df[['xmin', 'ymin', 'xmax', 'ymax']]
        return bbox

    data = model(img)
    bboxes = [make_bbox(im) for im in data.pandas().xyxy]
    return bboxes
def count_animals(old_bboxes, new_bboxes, image1, image2):
    """
    Count animals by difference between 2 dataframe with bboxes.
    It returns number of new animals in new image
    :param old_bboxes: pd.DataFrame
    :param new_bboxes: pd.DataFrame
    :return: int
    """
    def cond(x, y) -> bool:
        """
        return condition whether 2 bboxes are similar
        Similarity can be changed, but for now it is euclidean distance between bboxes less than threshold.
        :param x: iloc indexer of pd.DataFrame, indicating bbox
        :param y: iloc indexer of pd.DataFrame, indicating bbox
        :return: bool
        """
        threshold = 200
        x = torch.from_numpy(x.to_numpy())
        y = torch.from_numpy(y.to_numpy())
        s = torch.sum(torch.abs(x - y)).item()
        return s < threshold
    
    def tracking(traker, image1, image2, box: pd.DataFrame) -> bool:
        box = (box['xmin'], box['ymin'], box['xmax'] - box['xmin'], box['ymax'] - box['ymin'])
        frame1 = cv2.cvtColor(np.array(image1), cv2.COLOR_RGB2GRAY)
        tracker.init(frame1, box)
        frame2 = cv2.cvtColor(np.array(image2), cv2.COLOR_RGB2GRAY)
        status, bbox = traker.update(frame2)
        return status
    #temporary data structures that saves whether certain bbox in similar to bbox from previous image
    bbox = new_bboxes.copy()
    prev_bbox = old_bboxes.copy()
    prev_bbox['prev'] = False
    bbox['prev'] = False
    for i in range(len(old_bboxes)):
        for j in range(len(new_bboxes)):
            #if bbox is already similar to bbox from previous image, then we don't need to check this bbox
            if bbox.iloc[j]['prev'] or prev_bbox.iloc[i]['prev']:
                continue
            #similarity between bboxes from previous and present images
            if cond(old_bboxes.iloc[i], new_bboxes.iloc[j]):
                #save status of similarity
                prev_bbox['prev'][i] = True
                bbox['prev'][j] = True
                continue
            tracker = cv2.TrackerKCF_create()
            if tracking(tracker, image1, image2, old_bboxes.iloc[i].astype(int)):
                print('tracking')
                prev_bbox['prev'][i] = True
                bbox['prev'][j] = True
    #return number of non-similar(new animals) bboxes
    return bbox[bbox['prev'] == False].size.item()
def predict_sequence(images: list[Image.Image]):
    bboxes = predict_bboxes(images)
    #first image's bboxes are all animals
    count = bboxes[0].shape[0]
    for i in range(len(bboxes) - 1):
        #check the difference between two images and return new animals
        count += count_animals(bboxes[i], bboxes[i + 1], images[i], images[i + 1])
    return count
def predict(files: list[str]):
    """
    Prediction of number of animals in sequence of images.
    :param files: list[str], str is path to file
    :return: int
    """
    images = [Image.open(file) for file in files]
    return predict_sequence(images)

Algorithm of predicting number of animals in a sequence:
1. predict bboxes for all images using yolo model and filter low confidence predictions
2. iterate over results of 1. starting from first image of a sequence and do
2.1. number of bboxes in first image is count
2.2. for every next image we take bboxes of it and check if they are similar to bboxes from last image
2.3. similarity can be checked by coordinates of bboxes
2.4. if they are near bbox of last image they are the same animal from last image, otherwise this is new animal.
2.5. we can check if bbox is near using euclidian distance from every bbox of last image and mark it, if it is, so that we know that s
Using this algorithm we can say 'see' if animal is the same animal from last image if it didn't move much. But if it did move, we don't know for sure.

In [9]:
def run_seq(seq_id: str) -> int:
    """
    Run predictions for certain sequence providing seq_id.
    :param seq_id: sequence
    :return: int
    """
    seq = images[images['seq_id'] == seq_id]
    image_dir = '../../../ML/project/data/test/'
    files = [image_dir + file for file in seq.sort_values(by='seq_frame_num')['file_name'].tolist()]
    return predict(files)

In [10]:
run_seq('a91ebc18-0cd3-11eb-bed1-0242ac1c0002')

1

In [40]:
#run for all sequences in dataset
seqs = pd.DataFrame(data={
    'seq_id': sequences,
    'animal_count': np.vectorize(lambda x: run_seq(x))(sequences)}
)
#save result
seqs.to_csv('seqs.csv', sep='\t')

# Evaluate

In [ ]:
seq_data = pd.read_csv(DATA + "/seqs.csv",sep='\t',index_col=1)